In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

2023-12-26 10:21:03.628820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-26 10:21:04.474083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [4]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train, verbose=False)
candidate_recommender.fit(**candidate_recommender_study.best_params)

[I 2023-12-26 10:26:50,685] Using an existing study with name 'P3Beta' instead of creating a new one.


In [5]:
n_users, n_items = data_train.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [6]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe = training_dataframe.explode("ItemID")

In [7]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)

In [8]:
topPop = TopPop(data_train)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train)
p3alpha.fit(**p3alpha_study.best_params)

easer_study = optuna.create_study(
    study_name="Easer",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train)
easer.fit(**easer_study.best_params)

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train)
itemknncf.fit(**itemknncf_study.best_params)


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

TopPopRecommender: URM Detected 216 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 111 ( 0.5%) items with no interactions.


[I 2023-12-26 10:27:18,129] Using an existing study with name 'P3Alpha' instead of creating a new one.


P3alphaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 111 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4174.83 column/sec. Elapsed time 5.32 sec


[I 2023-12-26 10:27:32,361] Using an existing study with name 'Easer' instead of creating a new one.


EASE_R_Recommender: URM Detected 216 ( 1.7%) users with no interactions.
EASE_R_Recommender: URM Detected 111 ( 0.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 3.67 min


[I 2023-12-26 10:31:36,519] Using an existing study with name 'ItemKNNCF' instead of creating a new one.


ItemKNNCFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 111 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 10166.96 column/sec. Elapsed time 2.19 sec


In [9]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in range(n_users):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

item_popularity = np.ediff1d(sps.csc_matrix(data_train).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]

In [10]:
training_dataframe = training_dataframe.set_index('ItemID')
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "ItemID"})

In [11]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [19]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-26 10:45:20,789] A new study created in RDB with name: XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)


In [20]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [21]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_recs,
        train_labs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=1000)

[I 2023-12-26 10:46:36,741] Trial 0 finished with value: 0.06581843829560775 and parameters: {'objective': 'rank:map', 'n_estimators': 63, 'learning_rate': 0.08677511249344874, 'reg_alpha': 0.04130990220340621, 'reg_lambda': 0.044240191383192407, 'max_depth': 2, 'max_leaves': 2, 'grow_policy': 'depthwise', 'booster': 'gblinear'}. Best is trial 0 with value: 0.06581843829560775.
[I 2023-12-26 11:07:09,224] Trial 1 finished with value: 0.08588383700842542 and parameters: {'objective': 'rank:pairwise', 'n_estimators': 424, 'learning_rate': 0.0030004298323057666, 'reg_alpha': 0.052050455601365196, 'reg_lambda': 0.05041764407007444, 'max_depth': 4, 'max_leaves': 6, 'grow_policy': 'depthwise', 'booster': 'dart'}. Best is trial 1 with value: 0.08588383700842542.
[I 2023-12-26 11:07:58,180] Trial 2 finished with value: 0.0836599101214688 and parameters: {'objective': 'rank:ndcg', 'n_estimators': 64, 'learning_rate': 0.0013862427006259587, 'reg_alpha': 0.00027836637461264174, 'reg_lambda': 0.00

KeyboardInterrupt: 

In [1]:
study.best_params

NameError: name 'study' is not defined

In [ ]:
data_train_val = data_train + data_val

In [ ]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train_val, verbose=False)
candidate_recommender.fit(**candidate_recommender_study.best_params)

In [ ]:
n_users, n_items = data_train.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [ ]:
cutoff = 30

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe = training_dataframe.explode("ItemID")

In [ ]:
topPop = TopPop(data_train_val)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train_val)
p3alpha.fit(**p3alpha_study.best_params)

easer_study = optuna.create_study(
    study_name="Easer",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train_val)
easer.fit(**easer_study.best_params)

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train_val)
itemknncf.fit(**itemknncf_study.best_params)


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

In [ ]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in range(n_users):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

item_popularity = np.ediff1d(sps.csc_matrix(data_train_val).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train_val).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]

In [ ]:
training_dataframe = training_dataframe.drop("ItemID", axis=1)
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "ItemID"})

In [ ]:
training_dataframe["UserID"] = training_dataframe["UserID"].astype("category")
training_dataframe["ItemID"] = training_dataframe["ItemID"].astype("category")

In [ ]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [ ]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
)
XGB_model.fit(
    train_recs,
    train_labs,
    group=groups,
)
XGB_scores = XGB_model.predict(train_recs)
training_dataframe_copy = training_dataframe.copy()
training_dataframe_copy["XGB"] = XGB_scores

In [ ]:
def submission2(training_dataframe, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            try:
                user_recommendations = training_dataframe[training_dataframe["UserID"] == user_id]
                user_recommendations = user_recommendations.sort_values(by="XGB", ascending=False)
                item_list = np.array(user_recommendations["ItemID"].values.tolist()[:10])
                item_list = [itemmap[i] for i in item_list]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
            except IndexError:
                item_list = toppoprecommender.recommend(user_id, cutoff=10)
                item_list = [itemmap[i] for i in item_list]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
            except KeyError:
                item_list = toppoprecommender.recommend(user_id, cutoff=10)
                item_list = [itemmap[i] for i in item_list]
                print(item_list)
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(training_dataframe_copy, users, usermap, itemmap, data_train_val)